In [ ]:
!pip install flexible-fl opacus SciencePlots setuptools flexclash

In [ ]:
import copy
import os
import math
import torch
from flex.data import Dataset, FedDataDistribution, FedDataset, FedDatasetConfig
from flex.model import FlexModel
from flex.pool import FlexPool, fed_avg
from flex.pool.decorators import (
    deploy_server_model,
    init_server_model,
    set_aggregated_weights,
    collect_clients_weights,
)
from flexclash.data import data_poisoner
from flexclash.pool.defences import central_differential_privacy
import matplotlib.pyplot as plt
import matplotlib as mpl
import scienceplots
from typing import List
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
import opacus
from opacus import PrivacyEngine
from opacus.validators import ModuleValidator
from opacus.accountants.utils import get_noise_multiplier
from scipy.optimize import linprog
import pandas as pd

# --- CONSTANTS ---
ROUNDS = 100
EPOCHS = 1
N_CLIENTS = 10
POISONED= 1
fixed_epsilon = 1.0
fixed_delta = 0.001
budget = 100.0
delta_max = 0.1

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
def get_dataset():
    """
    Creation of the Federated MNIST dataset
    """
    train_data = MNIST(root=".", train=True, download=True, transform=None)
    test_data = MNIST(root=".", train=False, download=True, transform=None)
    flex_dataset = Dataset.from_torchvision_dataset(train_data)
    test_data = Dataset.from_torchvision_dataset(test_data)
    assert isinstance(flex_dataset, Dataset)

    config = FedDatasetConfig(seed=0)
    config.replacement = False
    config.n_nodes = N_CLIENTS

    flex_dataset = FedDataDistribution.from_config(flex_dataset, config)

    assert isinstance(flex_dataset, FedDataset)
    flex_dataset["server"] = test_data

    return flex_dataset

mnist_transforms = transforms.Compose(
    [transforms.ToTensor()]
)

In [ ]:
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(1024, 200)
        self.fc2 = nn.Linear(200, num_classes)

    def forward(self, x):
        x = self.pool(torch.tanh(self.conv1(x)))
        x = self.pool(torch.tanh(self.conv2(x)))
        x = x.view(-1, 1024)
        x = torch.tanh(self.fc1(x))
        x = self.fc2(x)
        return x

def get_model(num_classes=10):
  return ModuleValidator.fix(CNNModel(num_classes=num_classes))

In [ ]:
def train(client_flex_model: FlexModel, client_data: Dataset):
    """
    Train the model on the client data.
    """
    model = client_flex_model["model"]
    criterion = client_flex_model["criterion"]
    model.train()
    model = model.to(device)
    torch_dataset = client_data.to_torchvision_dataset(transform=mnist_transforms)
    optimizer = client_flex_model["optimizer_func"](model.parameters(), **client_flex_model["optimizer_kwargs"])
    dataloader = DataLoader(
        torch_dataset, batch_size=64, shuffle=True, pin_memory=False
    )

    for _ in range(EPOCHS):
        running_loss = 0.0
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

    return running_loss

def evaluate_model(server_flex_model: FlexModel, data):
    """
    Evaluate the model on the server data.
    """
    data = flex_dataset["server"]
    model = server_flex_model["model"]
    model.eval()
    test_loss = 0
    test_acc = 0
    total_count = 0
    model = model.to(device)
    criterion = server_flex_model["criterion"]

    test_dataset = data.to_torchvision_dataset(transform=mnist_transforms)
    test_dataloader = DataLoader(
        test_dataset, batch_size=256, shuffle=True, pin_memory=False
    )
    losses = []
    with torch.no_grad():
        for data, target in test_dataloader:
            total_count += target.size(0)
            data, target = data.to(device), target.to(device)
            output = model(data)
            losses.append(criterion(output, target).item())
            pred = output.data.max(1, keepdim=True)[1]
            test_acc += pred.eq(target.data.view_as(pred)).long().cpu().sum().item()

    test_loss = sum(losses) / len(losses)
    test_acc /= total_count
    return test_loss, test_acc

In [ ]:
# FLEX Decorators
@init_server_model
def build_server_model():
    server_flex_model = FlexModel()
    server_flex_model["model"] = get_model()
    server_flex_model["criterion"] = torch.nn.CrossEntropyLoss()
    server_flex_model["optimizer_func"] = torch.optim.SGD
    server_flex_model["optimizer_kwargs"] = {
        "lr": 1e-3,
        "weight_decay": 1e-7,
        "momentum": 0,
    }
    return server_flex_model

@deploy_server_model
def copy_server_model_to_clients(server_flex_model: FlexModel):
    new_flex_model = FlexModel()
    new_flex_model["model"] = copy.deepcopy(server_flex_model["model"])
    new_flex_model["server_model"] = copy.deepcopy(server_flex_model["model"])
    new_flex_model["discriminator"] = copy.deepcopy(server_flex_model["model"])
    new_flex_model["criterion"] = copy.deepcopy(server_flex_model["criterion"])
    new_flex_model["optimizer_func"] = copy.deepcopy(
        server_flex_model["optimizer_func"]
    )
    new_flex_model["optimizer_kwargs"] = copy.deepcopy(
        server_flex_model["optimizer_kwargs"]
    )
    return new_flex_model

@set_aggregated_weights
def set_agreggated_weights_to_server(server_flex_model: FlexModel, aggregated_weights):
    dev = aggregated_weights[0].get_device()
    dev = "cpu" if dev == -1 else "cuda"
    with torch.no_grad():
        weight_dict = server_flex_model["model"].state_dict()
        for layer_key, new in zip(weight_dict, aggregated_weights):
            weight_dict[layer_key].copy_(weight_dict[layer_key].to(dev) + new)

@collect_clients_weights
def get_clients_weights(client_flex_model: FlexModel):
    weight_dict = client_flex_model["model"].state_dict()
    server_dict = client_flex_model["server_model"].state_dict()
    dev = [weight_dict[name] for name in weight_dict][0].get_device()
    dev = "cpu" if dev == -1 else "cuda"
    return [
        (weight_dict[name] - server_dict[name].to(dev)).type(torch.float)
        for name in weight_dict
    ]

@data_poisoner
def label_flipping(img_array, label, num_classes=10):
    """Flips the label to a random different class."""
    while True:
        new_label = np.random.randint(0, num_classes)
        if new_label != label:
            break
    return img_array, new_label


In [ ]:
def run_attack_optimize_DP(pool: FlexPool):
    """
    Run the attack and optimize the epsilon and delta parameters. It saves the metrics in a csv file.
    """

    clients = pool.clients
    server = pool.servers

    epsilon_used = 0
    losses = []
    accuracies=[]
    epsilon_cummulative = []

    for i in range(ROUNDS):

        print(f"\n - Round {i+1}: Aggregating with with ε={fixed_epsilon:.3f}, δ={fixed_delta:.5f}")
        server.map(copy_server_model_to_clients, clients)

        epsilon_used += fixed_epsilon
        loss = clients.map(train)
        losses.append(loss[0])

        sensitivity = 0.01
        noise_multiplier = (sensitivity / fixed_epsilon) * np.sqrt(2 * np.log(1.25 / fixed_delta))
        print("Noise multiplier", noise_multiplier)

        pool.servers.map(get_clients_weights, clients)
        pool.servers.map(central_differential_privacy, l2_clip = 1.0, noise_multiplier = noise_multiplier)
        pool.servers.map(set_agreggated_weights_to_server, pool.servers)

        round_metrics = pool.servers.map(evaluate_model)
        accuracies.append(round_metrics[0][1]*100)
        print(" * Round metrics: ", round_metrics)
        epsilon_cummulative.append(epsilon_used)
        print(f"Epsilon used: {epsilon_used} \n")

        with open("experiment_DP_Static.txt", "a") as archivo:
            archivo.write(f"\n - Round {i+1}: Training with ε={fixed_epsilon:.3f}, δ={fixed_delta:.5f}\n")
            archivo.write(f"Round metrics: {round_metrics}\n")
            archivo.write(f"Epsilon used: {epsilon_used}\n")
            archivo.write("-" * 30 + "\n")

    df_metrics = pd.DataFrame({
      'Round': list(range(1, ROUNDS + 1)),
      'Accuracy (%)': accuracies,
      'Epsilon Acumulado': epsilon_cummulative
      })
    df_metrics.to_csv('metrics_DP_Static.csv', index=False)

In [ ]:
if __name__ == "__main__":
    flex_dataset = get_dataset()
    poisoned_clients_ids = list(flex_dataset.keys())[:POISONED]
    flex_dataset = flex_dataset.apply(label_flipping, node_ids=poisoned_clients_ids)
    pool = FlexPool.client_server_pool(
        fed_dataset=flex_dataset, init_func=build_server_model
    )
    run_attack_optimize_DP(pool)